This is a network constructor.

Import modules

In [2]:
# This line configures matplotlib to show figures embedded in the notebook, 
# instead of opening a new window for each figure. More about that later. 
# If you are using an old version of IPython, try using '%pylab inline' instead.
%matplotlib inline
%load_ext snakeviz

import numpy as np
from scipy.optimize import minimize
from scipy.special import expit
import matplotlib.pyplot as plt

from matplotlib.lines import Line2D

import timeit

import pandas as pd

import plotly.plotly as py
from plotly.graph_objs import *
import plotly.tools as tls

The snakeviz extension is already loaded. To reload it, use:
  %reload_ext snakeviz


Predefine useful constants

In [3]:
# hbar=1.054571726*10**(-34)
hbar=1
# delm2E=1
# lamb=1  ## lambda for neutrinos
# lambb=1 ## lambda for anti neutrinos
# gF=1
# nd=1  ## number density
# ndb=1   ## number density
omega=1
omegab=-1

## Here are some matrices to be used

elM = np.array([[1,0],[0,0]])
bM = 1/2*np.array( [ [ - 0.38729833462,0.31622776601] , [0.31622776601,0.38729833462] ] )

## sqareroot of 2
sqrt2=np.sqrt(2)

In [4]:
def actf(x):
    #return 1/(1+np.exp(-x)) # It's not bad to define this function here for people could use other functions other than expit(x).
    return expit(x)

In [5]:
## Very important notes
## fOft(x,ti) should be specified, here ti is a scalar value not a list. and it should return a value.
## Here I use t as the variables list and ti as the variable.



def costODE(x,t,initialCondition,fOfArg):   # x is a list of the order v,w,u. x will be splited to three equal parts.
    # initialCondition can only be constants.

    t = np.array(t)
    
    costODETotal = np.sum( costODETList(x,t,initialCondition,fOfArg)  )
        
    return costODETotal
    

def costODETList(x,t,initialCondition,fOfArg):  ## This is the function WITHOUT the square!!! 
    
    v,w,u = np.split(x,3)[:3]
    
    t = np.array(t)
    
    costList = np.asarray([])
    
    for temp in t:
        tempElement = costODETi(x,temp,initialCondition,fOfArg)
        costList = np.append(costList, tempElement)
        
    return np.array(costList)

    

def costODETi(x,ti,initialCondition,fOfArg):  # function for each t. here t is a single value 
    # fOfArg is the function f(t) in the example

    v,w,u = np.split(x,3)[:3]
    
    args = np.array([x,ti,initialCondition])
    
    fvec = np.array(actf(ti*w + u) )  # This is a vector!!!
    ft = fOfArg(args)  ## fOft should be specified in a problem!!!!!!!!!!!! And it takes a whole array of all the arguments
    # For a given t, this calculates the value of y(t), given the parameters, v, w, u.  Notice this initialCondition.
    
    return  ( np.sum (v*fvec + ti * v* fvec * ( 1 -  fvec  ) * w ) + ft )   ** 2
    

    
## The funNNi(x,ti,initialCondition) takes a time ti and exports the function value with input x.

    
def funNNi(x,ti,initialCondition):  # for a single time stamp t
    
    v,w,u = np.split(x,3)[:3]
    
    return initialCondition +  np.sum(ti * v * actf( ti*w +u ) )

## funNNList(x,t,initialCondition) takes a list of time and exports the function values of these times.

def funNNList(x,t,initialCondition):
    
    t = np.array(t)
    
    tempList = np.asarray([])
    
    for ti in t:
        tempElement = funNNi(x,ti,initialCondition)
        tempList = np.append(tempList,tempElement)
        
    return np.array(tempList)